In [1]:
import requests
import json
import random
import time
import pickle


model = "dRIT"

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


# TRAIN de SNLI

In [2]:
import pandas as pd

In [ ]:
df =pd.read_pickle("predictions/RL_SNLI_TEST.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases
0,2,0.750000,0.0,0.250000,0.400892,0.062678,3.927563,0.309673,0.500000,1,0,0.450000,0,0,0.250000,neutral
1,2,0.333333,0.0,0.500000,0.534522,0.200153,4.265780,0.695562,0.666667,0,0,0.566667,0,0,0.666667,entailment
2,2,1.000000,0.0,0.333333,0.239046,0.095411,3.913399,0.392654,0.250000,0,0,0.433333,0,0,0.000000,contradiction
3,1,0.000000,0.0,0.500000,0.384900,0.117440,4.381176,0.644971,1.000000,1,0,0.481481,0,0,1.000000,neutral
4,1,0.000000,0.0,0.500000,0.384900,0.127250,4.413306,0.778108,0.500000,0,1,0.481481,0,0,1.000000,entailment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9813,2,0.750000,0.0,0.333333,0.516398,0.058824,4.233124,0.534993,0.400000,0,0,0.511111,0,0,0.250000,contradiction
9814,-1,1.000000,0.5,0.333333,0.333333,0.221384,4.141572,0.700668,1.000000,2,0,0.694444,0,0,0.000000,entailment
9815,2,0.666667,0.0,0.333333,0.333333,0.038890,4.086125,0.389466,0.333333,0,0,0.453704,0,0,0.333333,contradiction
9816,1,0.000000,0.0,1.000000,0.666667,0.152691,4.673955,1.000000,1.000000,0,0,0.490741,0,0,1.000000,entailment


In [4]:
original =pd.read_csv("../RIT_ollama/SNLI/TEST.csv")
original

,Unnamed: 0,gold_label,sentence1,sentence2,SET
0,549367,neutral,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,TEST
1,549368,entailment,This church choir sings to the masses as they ...,The church is filled with song.,TEST
2,549369,contradiction,This church choir sings to the masses as they ...,A choir singing at a baseball game.,TEST
3,549370,neutral,"A woman with a green headscarf, blue shirt and...",The woman is young.,TEST
4,549371,entailment,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,TEST
...,...,...,...,...,...
9813,559186,contradiction,Two women are observing something together.,Two women are standing with their eyes closed.,TEST
9814,559187,entailment,Two women are observing something together.,Two girls are looking at something.,TEST
9815,559188,contradiction,A man in a black leather jacket and a book in ...,A man is flying a kite.,TEST
9816,559189,entailment,A man in a black leather jacket and a book in ...,A man is speaking in a classroom.,TEST


In [5]:
df["sentence_A"]=original["sentence1"]
df["sentence_B"]=original["sentence2"]

In [6]:
entailment=df[df["clases"]=="entailment"]
neutral=df[df["clases"]=="neutral"]
contradiction=df[df["clases"]=="contradiction"]
contradiction

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
2,2,1.000000,0.0,0.333333,0.239046,0.095411,3.913399,0.392654,0.250000,0,0,0.433333,0,0,0.000000,contradiction,This church choir sings to the masses as they ...,A choir singing at a baseball game.
5,2,0.500000,0.0,0.000000,0.384900,0.074209,4.440310,0.539767,0.500000,0,0,0.481481,0,0,0.500000,contradiction,"A woman with a green headscarf, blue shirt and...",The woman has been shot.
8,2,0.666667,0.0,0.250000,0.408248,0.125148,4.139124,0.659479,0.666667,1,0,0.694444,0,0,0.333333,contradiction,An old man with a package poses in front of an...,A man walks by an ad.
11,2,0.750000,0.0,0.250000,0.447214,0.101787,4.040790,0.337186,0.250000,0,0,0.483333,0,0,0.250000,contradiction,A statue at a museum that no seems to be looki...,Tons of people are gathered around the statue.
14,2,0.750000,0.0,0.333333,0.400000,0.043631,4.145050,0.409257,0.250000,0,0,0.600000,0,0,0.250000,contradiction,A land rover is being driven across a river.,A sedan is stuck in the middle of a river.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9804,2,0.333333,0.0,0.666667,0.516398,0.231151,4.449800,0.765826,0.666667,0,0,0.688889,0,0,0.666667,contradiction,A boy drags his sled through the snow.,a boy drags his dog
9807,2,0.750000,0.0,0.250000,0.372104,0.071262,4.108551,0.518640,0.400000,0,0,0.434615,0,0,0.250000,contradiction,A man wearing glasses and a ragged costume is ...,Two men are playing frisbee in the park.
9810,2,1.000000,0.0,0.000000,0.000000,0.012453,3.721939,0.112340,0.250000,1,0,0.280303,0,0,0.000000,contradiction,A female dancer wearing all white dancing whil...,Some players are playing cricket in ground
9813,2,0.750000,0.0,0.333333,0.516398,0.058824,4.233124,0.534993,0.400000,0,0,0.511111,0,0,0.250000,contradiction,Two women are observing something together.,Two women are standing with their eyes closed.


In [ ]:
df_e=entailment.sample(n=200)
df_n=neutral.sample(n=200)
df_c=contradiction.sample(n=200)
df_t=pd.concat([df_e,df_n,df_c])
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
5395,1,0.000000,0.0,1.000000,0.666667,0.182158,4.758932,1.000000,1.000000,0,0,0.773148,0,0,1.000000,entailment,A small child with a backpack walks down a nar...,A small child is walking on a road
7386,2,0.333333,0.0,1.000000,0.589256,0.140694,4.613125,1.000000,1.000000,0,0,0.464286,0,0,0.666667,entailment,A brown and white dog walking away from a blac...,A dog stands in the water.
3686,2,0.666667,0.0,0.333333,0.288675,0.108599,4.038265,0.681953,0.666667,1,0,0.555556,0,0,0.333333,entailment,A man in the bucket of a chery-picker cuts dow...,A man is trimming trees.
7528,2,0.666667,0.0,0.500000,0.377964,-0.049452,4.145515,0.535615,0.666667,1,0,0.464286,0,0,0.333333,entailment,Three people are making transactions at an aut...,A line of people at an ATM.
8594,2,0.666667,0.0,0.000000,0.154303,0.108264,3.879840,0.510767,1.000000,2,0,0.436508,0,0,0.333333,entailment,A woman looks at a picture and framing tools w...,There is a person in a room.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131,2,0.400000,0.0,0.750000,0.894427,0.231952,4.379169,0.776339,0.600000,0,0,0.933333,0,0,0.600000,contradiction,There are two people walking in the sand.,There are five people walking in the sand with...
7947,2,0.333333,0.0,0.400000,0.524142,0.164122,4.365030,0.806454,1.000000,2,0,0.543040,0,0,0.666667,contradiction,A man with a gray beard and a little boy are s...,The man and the boy are taking a nap on the bu...
9169,2,0.750000,0.0,0.333333,0.404520,0.105212,4.266216,0.610287,0.750000,1,0,0.263636,0,0,0.250000,contradiction,"A race car, numbered 104 is rounding a bend as...",People are watching a ball game.
7738,2,1.000000,0.0,0.000000,0.154303,-0.003144,3.861699,0.143365,0.250000,1,0,0.311111,0,0,0.000000,contradiction,A woman in a white dress with a white umbrella...,Two sisters are flipping each other off.


In [8]:
lista_respuestasOllama=[]

In [9]:
df.columns

Index(['relation', 'no_matcheadas', 'contradiction', 'jaccard', 'simBoW',
       'sumas', 'mutinf_t', 'max_info_t', 'entail', 'list_comp', 'list_incomp',
       'Jaro-Winkler_rit', 'negT', 'negH', 'overlap_ent', 'clases',
       'sentence_A', 'sentence_B'],
      dtype='object')

In [10]:
faltantes=[]

In [ ]:
df_t=df_t.reset_index(drop=True)
df_t

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,1,0.000000,0.0,1.000000,0.666667,0.182158,4.758932,1.000000,1.000000,0,0,0.773148,0,0,1.000000,entailment,A small child with a backpack walks down a nar...,A small child is walking on a road
1,2,0.333333,0.0,1.000000,0.589256,0.140694,4.613125,1.000000,1.000000,0,0,0.464286,0,0,0.666667,entailment,A brown and white dog walking away from a blac...,A dog stands in the water.
2,2,0.666667,0.0,0.333333,0.288675,0.108599,4.038265,0.681953,0.666667,1,0,0.555556,0,0,0.333333,entailment,A man in the bucket of a chery-picker cuts dow...,A man is trimming trees.
3,2,0.666667,0.0,0.500000,0.377964,-0.049452,4.145515,0.535615,0.666667,1,0,0.464286,0,0,0.333333,entailment,Three people are making transactions at an aut...,A line of people at an ATM.
4,2,0.666667,0.0,0.000000,0.154303,0.108264,3.879840,0.510767,1.000000,2,0,0.436508,0,0,0.333333,entailment,A woman looks at a picture and framing tools w...,There is a person in a room.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,0.400000,0.0,0.750000,0.894427,0.231952,4.379169,0.776339,0.600000,0,0,0.933333,0,0,0.600000,contradiction,There are two people walking in the sand.,There are five people walking in the sand with...
596,2,0.333333,0.0,0.400000,0.524142,0.164122,4.365030,0.806454,1.000000,2,0,0.543040,0,0,0.666667,contradiction,A man with a gray beard and a little boy are s...,The man and the boy are taking a nap on the bu...
597,2,0.750000,0.0,0.333333,0.404520,0.105212,4.266216,0.610287,0.750000,1,0,0.263636,0,0,0.250000,contradiction,"A race car, numbered 104 is rounding a bend as...",People are watching a ball game.
598,2,1.000000,0.0,0.000000,0.154303,-0.003144,3.861699,0.143365,0.250000,1,0,0.311111,0,0,0.000000,contradiction,A woman in a white dress with a white umbrella...,Two sisters are flipping each other off.


In [12]:
df_t.to_pickle("predictions/RL_SNLI_TEST_600.pickle")

# RIT base

In [14]:
for i in range(1):
    for index,strings in df_t[i*600:(i+1)*600].iterrows():
        prompt = ''''Analyze the following sentences: 
        Text: '''+strings["sentence_A"]+'''
        Hypothesis: '''+ strings["sentence_B"]+'''
        only responds using the template. '''
        #print(prompt)
        data = {
            "prompt": prompt,
            "model": model,
            "format": "json",
            "stream": False,
            #"options": {"temperature": 0.2},
        }
        try:
            response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)        
            json_data = json.loads(response.text)
            lista_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
            print(index)
        except:
            print("Saltó",index)
            faltantes.append(index)
            lista_respuestasOllama.append("NA")
    with open("resultados/rit_base_SNLI_TEST_600_"+str(i+1)+".pickle", "wb") as f:
        pickle.dump(lista_respuestasOllama, f)
    print("Primer rango: ",(i)*600,(i+1)*600)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
print(len(lista_respuestasOllama))

600
